**Importing various libraries**

In [1]:
pip install -U scikit-learn

In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import _pickle as cPickle
from scipy.io import loadmat
from sklearn.svm import SVC
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')
from google.colab import drive
drive.mount('/content/drive')

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, f1_score, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve
from sklearn.naive_bayes import MultinomialNB

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/MyDrive/SEM6/Prak. Pembelajaran Mesin/Pertemuan9/Training.txt",sep="\t", names=['liked','text'],encoding="utf-8");
df.head(3)
#df

,liked,text
0,1,India is developing countries
1,1,The Da Vinci Code book is just awesome.
2,1,this was the first clive cussler i've ever rea...


In [4]:
print(len(df))

6931


In [5]:
df.groupby('liked').describe()

text                                  
      count unique                   top freq
liked                                        
0      2975    559  I hate Harry Potter.   85
1      3956    744  I love Harry Potter.  167

# Data preprocessing

In [8]:
def tokens(review):
    return TextBlob(review).words

In [9]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
df.head().text.apply(tokens)

,text
0,"[India, is, developing, countries]"
1,"[The, Da, Vinci, Code, book, is, just, awesome]"
2,"[this, was, the, first, clive, cussler, i, 've..."
3,"[i, liked, the, Da, Vinci, Code, a, lot]"
4,"[i, liked, the, Da, Vinci, Code, a, lot]"


Fungsi token() dibuat untuk mengurai data/review menjadi kata-kata.

In [12]:
nltk.download('averaged_perceptron_tagger_eng')
TextBlob("ready was not a good movie").tags

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


[('ready', 'NN'),
 ('was', 'VBD'),
 ('not', 'RB'),
 ('a', 'DT'),
 ('good', 'JJ'),
 ('movie', 'NN')]

In [14]:
nltk.download('wordnet')

def to_lemmas(review):
    wordss = TextBlob(review.lower()).words
    # for each word, take its "base form" = lemma
    return [word.lemma for word in wordss]

df.text.head().apply(to_lemmas)

[nltk_data] Downloading package wordnet to /root/nltk_data...


,text
0,"[india, is, developing, country]"
1,"[the, da, vinci, code, book, is, just, awesome]"
2,"[this, wa, the, first, clive, cussler, i, 've,..."
3,"[i, liked, the, da, vinci, code, a, lot]"
4,"[i, liked, the, da, vinci, code, a, lot]"



Lemmatization adalah salah satu tahapan penting dalam preprocessing data. Pada tahapan ini, kata-kata diubah menjadi lema (bentuk dasar). Sebagai contoh “octopi” diubah menjadi “gurita”. metode yang mirip dengan stemming.

NLTK juga menyediakan lemmatizer yang sangat kuat yang memanfaatkan WORDNET, misalnya.

In [15]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('octopi')
#nltk

'octopus'

# Converting text data into vectors

In [16]:
bow_transformer = CountVectorizer(analyzer=to_lemmas).fit(df['text'])
print(len(bow_transformer.vocabulary_))

2114



Sklearn menawarkan Countvectorizer yang menghitung frekuensi kata tertentu dalam dokumen. Ini memberikan nomor unik untuk setiap kata dalam koleksi.

In [17]:
review1=df['text'][3]
print(review1)
#to check 3rd document/review in collection/database

i liked the Da Vinci Code a lot.


In [18]:
bow=bow_transformer.transform([review1])
print(bow)
bow.shape

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 8 stored elements and shape (1, 2114)>
  Coords	Values
  (0, 42)	1
  (0, 369)	1
  (0, 458)	1
  (0, 950)	1
  (0, 1123)	1
  (0, 1152)	1
  (0, 1838)	1
  (0, 1977)	1


(1, 2114)


Countvectorizer membuat matriks yang jarang.

In [20]:
print(bow_transformer.get_feature_names_out()[372]) # Use get_feature_names_out() instead of get_feature_names()
#to check 372nd word in collection

code-other


In [21]:
review_bow = bow_transformer.transform(df['text'])
print( 'sparse matrix shape:', review_bow.shape)
print('number of non-zeros:', review_bow.nnz) #learn this
print( 'sparsity: %.2f%%' % (100.0 * review_bow.nnz))

sparse matrix shape: (6931, 2114)
number of non-zeros: 71297
sparsity: 7129700.00%



Countvectorizer membuat matriks jarang.bentuk matriks jarang n*m di mana n adalah total dokumen dan m adalah total kata unik.


CountVectorizer hanya menghitung frekuensi kata dalam dokumen tersebut. Tetapi sering kali beberapa kata seperti (the, or) muncul banyak kali dalam koleksi yang sebenarnya tidak berkontribusi dalam menentukan polaritas dokumen tertentu sehingga untuk meniadakan efeknya, metode pembobotan khusus digunakan.

# Tf-idf Vectorizer


Tf berarti term-frequency sedangkan tf-idf berarti term-frequency dikalikan invers document-frequency.

Tujuan menggunakan tf-idf daripada hanya CountVectorizer dalam dokumen tertentu adalah untuk mengurangi dampak token yang sangat sering muncul dalam korpus tertentu (yang kurang informatif) daripada token yang muncul beberapa kali.

Tf-idf(d,t)=tf(d,t)*idf(d,t);

dimana tf(d,t) adalah term frequency yang menyatakan berapa kali kata/token t muncul di dalam dokumen tersebut dibagi dengan jumlah total kata di dalam dokumen tersebut.

dan idf(d,t)=log[n/(df(d,t))] yaitu jumlah total dokumen dibagi dengan jumlah dokumen yang mengandung kata/token t.

In [22]:
tfidf_transformer =TfidfTransformer().fit(review_bow)
review_tfidf = tfidf_transformer.transform(review_bow)
review_tfidf.shape

(6931, 2114)


Count vectorizer memberikan keluaran berupa frekuensi kata-kata yang berbeda dalam korpus kita, yang kemudian diteruskan ke metode transformasi tf-idf_transformer.

Ini mentransformasikan matriks hitungan ke representasi tf atau tf-idf yang dinormalisasi

In [23]:
text_train, text_test, liked_train, liked_test = train_test_split(df['text'], df['liked'], test_size=0.2)
print(len(text_train), len(text_test), len(text_train) , len(text_test))

5544 1387 5544 1387



Dataset yang diunduh kemudian dibagi menjadi data latih dan data uji dengan rasio 0,8 banding 0,2

In [24]:
pipeline_svm = Pipeline([
    ('bow', CountVectorizer(analyzer=to_lemmas)),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC()),
])


Pipeline secara berurutan menerapkan daftar transformasi dan estimator akhir. Langkah-langkah perantara dari pipeline harus berupa ‘transformasi’, yaitu harus menerapkan metode fit dan transformasi. Estimator akhir hanya perlu mengimplementasikan fit.

Tujuan dari pipeline adalah untuk mengumpulkan beberapa langkah yang dapat divalidasi silang secara bersamaan sambil menetapkan parameter yang berbeda.

In [25]:
# pipeline parameters to automatically explore and tune
param_svm = [
  {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
  {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf']},
]

**SVM**

In [28]:
grid_svm = GridSearchCV(
    pipeline_svm,
    param_grid=param_svm,
    refit=True,
    n_jobs=-1,
    scoring='accuracy',
    cv=StratifiedKFold(n_splits=5),
)


Pencarian menyeluruh atas nilai parameter yang ditentukan untuk sebuah estimator.

CV adalah singkatan dari validasi silang. Mempelajari parameter fungsi prediksi dan mengujinya pada data yang sama adalah kesalahan metodologis, hal ini akan selalu memberikan akurasi 100% dan oleh karena itu data pelatihan dan pengujian harus berbeda. Validasi silang adalah ide untuk membagi data pelatihan menjadi k lipatan yaitu k subset. Prosedur berikut ini diikuti untuk setiap k “fold”:

Sebuah model dilatih dengan menggunakan k-1 dari folds sebagai data pelatihan;

Model yang dihasilkan divalidasi pada bagian data yang tersisa (yaitu, digunakan sebagai set pengujian untuk menghitung ukuran kinerja seperti akurasi).

In [30]:
%time classifier = grid_svm.fit(text_train, liked_train) # find the best combination from param_svm
# Access the results using cv_results_ instead of grid_scores_
print(classifier.cv_results_)

CPU times: user 3.01 s, sys: 224 ms, total: 3.23 s
Wall time: 2min 23s
{'mean_fit_time': array([2.93350735, 2.21106935, 2.77329116, 2.21042023, 6.33782072,
       6.2300736 , 4.36579833, 6.26989217, 2.33368726, 5.75402026,
       2.50050511, 2.56854653]), 'std_fit_time': array([0.41147904, 0.12818225, 0.70491072, 0.05686795, 0.78498659,
       0.85809856, 0.71671436, 0.90793285, 0.03930452, 1.61550293,
       0.43478882, 0.31109842]), 'mean_score_time': array([0.66583338, 0.51651225, 0.67007031, 0.52229333, 1.10932431,
       1.2609602 , 0.97974577, 1.09732447, 0.59130831, 1.16317053,
       0.60885115, 0.66842046]), 'std_score_time': array([0.18404829, 0.01832509, 0.1658548 , 0.00885445, 0.0534536 ,
       0.24072675, 0.07643578, 0.07242033, 0.01833061, 0.32607716,
       0.15309682, 0.13200657]), 'param_classifier__C': masked_array(data=[1, 10, 100, 1000, 1, 1, 10, 10, 100, 100, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False,
                  

In [31]:
print(classification_report(liked_test, classifier.predict(text_test)))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       585
           1       0.99      1.00      0.99       802

    accuracy                           0.99      1387
   macro avg       0.99      0.99      0.99      1387
weighted avg       0.99      0.99      0.99      1387



In [32]:
print(classifier.predict(["the vinci code is awesome"])[0])

1


In [33]:
print(classifier.predict(["the vinci code is bad"])[0])

0


In [34]:
def gaussKernel(x1, x2, sigma):
    ss=np.power(sigma,2)
    norm= (x1-x2).T.dot(x1-x2)
    return np.exp(-norm/(2*ss))
x1 = np.array([1, 2, 1])
x2 = np.array([0, 4, -1])
sigma = 2
gaussKernel(x1,x2,sigma)

np.float64(0.32465246735834974)